In [1]:
import pandas as pd
import numpy as np
import os
import sys

In [2]:
reports = pd.read_excel('../data/financial_reports/2019년12월확정실적/201912(한글).xlsx', sheet_name='주재무제표', header=[1, 2], dtype={'산업코드':np.str})

In [3]:
reports.head()

시장               종목코드                회사명               회계기준  \
  Unnamed: 0_level_1 Unnamed: 1_level_1 Unnamed: 2_level_1 Unnamed: 3_level_1   
0                 KS            A000020               동화약품                 연결   
1                 KS            A000040              KR모터스                 연결   
2                 KS            A000050                 경방                 연결   
3                 KS            A000060              메리츠화재                 연결   
4                 KS            A000070              삼양홀딩스                 연결   

                산업코드                산업명                 단위         자산총계  \
  Unnamed: 4_level_1 Unnamed: 5_level_1 Unnamed: 6_level_1       201912   
0              32100   의료용 물질 및 의약품 제조업                 천원    376028553   
1              33100        기타 운송장비 제조업                 천원    144081160   
2              74700        소매업; 자동차 제외                 천원   1370742491   
3             116601                보험업                 천원  23215663095   
4             137100            전문 서비스업                 천원   3589513891   

                               ... 판매비와관리비\n증가율(YoY) 발표영업이익\n증가율(YoY)  \
     201812(비교)        201909  ...            201912           201912   
0  3.705992e+08  3.747781e+08  ...           6.81762         -11.6521   
1  1.549011e+08  1.272469e+08  ...            51.432               적확   
2  1.301793e+09  1.334184e+09  ...            1.5952         -21.8626   
3  2.047876e+10  2.307962e+10  ...               NaN              NaN   
4  3.313797e+09  3.667812e+09  ...           11.7581         -42.6712   

  법인세비용차감전이익\n증가율(YoY) 계속사업이익\n증가율(YoY) 순이익증가율(YoY) 지배주주순이익\n증가율(YoY)  \
                201912           201912      201912            201912   
0              14.2293         -6.69696    -6.69696          -6.61742   
1                   적확               적확          적확                적확   
2              98.5454          116.064     116.064           115.993   
3                  NaN              NaN         NaN               NaN   
4             -37.3429         -39.4766    -39.4766          -55.8799   

   발표영업이익률  계속사업이익률     순이익률 지배주주순이익률  
    201912   201912   201912   201912  
0  3.22896  3.05839  3.05839  3.06112  
1  -18.593 -30.9563 -30.9563 -30.5316  
2  9.31121  12.6847  12.6847  12.6833  
3      NaN      NaN      NaN      NaN  
4  3.18964  2.49316  2.49316  1.24058  

[5 rows x 261 columns]

In [4]:
columns = reports.columns.values
selected = []
for tp in columns:
    if tp[0] in ['시장', '종목코드', '회사명', '산업코드', '산업명', '단위', '자산총계', '유동자산', '부채총계', '자본총계', '매출액', '매출원가', '매출총이익', '당기순이익', '유동비율', '부채비율', '영업활동으로인한\n현금흐름'] \
    and (tp[1] == '201912' or tp[1].startswith('Unnamed:')):
         selected.append(tp)
df = reports.loc[:, selected]
new_cols = [el[0] for el in selected]
new_cols[14] = '영업현금흐름'
df.columns = new_cols

In [5]:
# df[~df['종목코드'].str.startswith('A')]
df['종목코드'] = df['종목코드'].str[1:]

In [6]:
df.head()

,시장,종목코드,회사명,산업코드,산업명,단위,자산총계,유동자산,부채총계,자본총계,매출액,매출원가,매출총이익,당기순이익,영업현금흐름,유동비율,부채비율
0,KS,000020,동화약품,32100,의료용 물질 및 의약품 제조업,천원,376028553,2.354570e+08,75153959,300874594,3.071500e+08,1.854183e+08,121731711.0,9393855,1.328497e+07,422.612518,24.978500
1,KS,000040,KR모터스,33100,기타 운송장비 제조업,천원,144081160,5.269909e+07,108871143,35210017,1.325793e+08,1.335669e+08,-987674.0,-41041685,-7.078682e+06,61.021527,309.205028
2,KS,000050,경방,74700,소매업; 자동차 제외,천원,1370742491,2.648610e+08,629189292,741553199,3.438921e+08,2.443960e+08,99496064.0,43621631,6.687528e+07,137.675752,84.847492
3,KS,000060,메리츠화재,116601,보험업,천원,23215663095,NaN,20775120387,2440542707,NaN,NaN,NaN,301296160,1.026132e+09,NaN,NaN
4,KS,000070,삼양홀딩스,137100,전문 서비스업,천원,3589513891,1.259478e+09,1560007367,2029506524,2.488563e+09,2.025132e+09,463430768.0,62043854,2.189381e+08,238.930751,76.866339


In [7]:
price_date = '200414'
stocks = pd.read_csv('../data/stock_prices/20%s.csv'%price_date)
# kospi.columns = ['종목코드', '종목명', '현재가', '']

In [8]:
num_cols = ['현재가', '대비', '거래량', '거래대금', '시가', '고가', '저가', '시가총액', '상장주식수']
for col in num_cols:
    stocks[col] = stocks[col].str.replace(",", "").astype(np.int64)
    
stocks1 = stocks[['순위', '종목코드', '종목명', '현재가', '거래량', '시가총액', '상장주식수']]
print(len(stocks1))
stocks1.head()

2476


,순위,종목코드,종목명,현재가,거래량,시가총액,상장주식수
0,1,005930,삼성전자,49000,14206216,292519344950000,5969782550
1,2,000660,SK하이닉스,82300,3344109,59914594639500,728002365
2,3,207940,삼성바이오로직스,542000,539127,35861430000000,66165000
3,4,005935,삼성전자우,42100,1193998,34643530070000,822886700
4,5,068270,셀트리온,216000,1273985,27722869704000,128346619


In [9]:
len(stocks)

2476

In [10]:
bad_set_1 = set(pd.read_csv('../data/관리종목.csv', dtype={'종목코드':np.str})['종목코드'])
bad_set_2 = set(pd.read_csv('../data/매매거래정지.csv', dtype={'종목코드':np.str})['종목코드'])
bad_set_3 = set(pd.read_csv('../data/정리매매종목.csv', dtype={'종목코드':np.str})['종목코드'])
bad_set_4 = set(pd.read_csv('../data/투자경고종목.csv', dtype={'종목코드':np.str})['종목코드'])
bad_sets = bad_set_1 | bad_set_2 | bad_set_3 | bad_set_4
print(len(bad_sets))

167


In [11]:
stocks1[stocks1['종목코드'] == '035420']

,순위,종목코드,종목명,현재가,거래량,시가총액,상장주식수
5,6,035420,NAVER,168500,520725,27678382057500,164263395


In [12]:
stocks1[stocks1['종목코드'] == '035420']['시가총액']

5    27678382057500
Name: 시가총액, dtype: int64

In [13]:
stocks2 = stocks1[~stocks1['종목코드'].isin(bad_sets)]
stocks2 = stocks2[~stocks2['종목명'].str.endswith('우')]
stocks2 = stocks2[~stocks2['종목명'].str.endswith('우B')]
stocks2 = stocks2[~stocks2['종목명'].str.contains('스팩')]
stocks2 = stocks2[stocks2['거래량'] > 1000]
stocks2 = stocks2[stocks2['현재가'] > 1000]
len(stocks2)

1886

In [14]:
# joined = stocks2.set_index('종목코드').join(df.set_index('종목코드'), on='종목코드', how='inner')
joined = pd.merge(stocks2, df, on='종목코드')
joined

,순위,종목코드,종목명,현재가,거래량,시가총액,상장주식수,시장,회사명,산업코드,...,유동자산,부채총계,자본총계,매출액,매출원가,매출총이익,당기순이익,영업현금흐름,유동비율,부채비율
0,1,005930,삼성전자,49000,14206216,292519344950000,5969782550,KS,삼성전자,32600,...,1.813853e+11,89684076000,262880421000,2.304009e+11,1.472395e+11,8.316133e+10,21738865000,4.538292e+10,284.379742,34.115921
1,2,000660,SK하이닉스,82300,3344109,59914594639500,728002365,KS,SK하이닉스,32600,...,1.445760e+10,16846299000,47943195000,2.699073e+10,1.882528e+10,8.165458e+09,2016391000,6.483188e+09,183.611143,35.138040
2,3,207940,삼성바이오로직스,542000,539127,35861430000000,66165000,KS,삼성바이오로직스,32100,...,1.356262e+09,1557172767,4354453745,7.015919e+08,5.095844e+08,1.920074e+08,202904099,1.015362e+07,NaN,NaN
3,5,068270,셀트리온,216000,1273985,27722869704000,128346619,KS,셀트리온,32100,...,1.787340e+09,986733418,2906958188,1.128460e+09,4.984397e+08,6.300202e+08,297969203,4.739939e+08,271.952220,33.943846
4,6,035420,NAVER,168500,520725,27678382057500,164263395,KS,NAVER,106300,...,5.643714e+09,5795601052,6503926069,6.593400e+09,NaN,6.593400e+09,396821062,1.356843e+09,149.576907,89.109270
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1780,2271,054410,케이피티유,3020,13086,15281200000,5060000,KQ,케이피티유,32500,...,1.180718e+07,29344083,32888826,2.426961e+07,2.122144e+07,3.048165e+06,-764474,2.484600e+06,NaN,NaN
1781,2276,096870,엘디티,2200,21496,14692480000,6678400,KQ,엘디티,32600,...,5.667297e+06,1943262,9881656,6.955660e+06,5.057515e+06,1.898145e+06,-1398661,1.221650e+05,NaN,NaN
1782,2278,025890,한국주강,1290,7715,14588944110,11309259,KS,한국주강,32900,...,1.283543e+07,16295373,32212273,3.627140e+07,3.844710e+07,-2.175700e+06,-13564456,3.071580e+05,NaN,NaN
1783,2296,019490,하이트론,2305,27489,12748344175,5530735,KS,하이트론,32600,...,3.383360e+07,38483318,17097678,3.469537e+07,3.819625e+07,-3.500882e+06,-17825443,-8.217431e+06,126.450375,225.079207


In [15]:
# joined[['순위', '종목코드', '종목명', '현재가', '산업코드', '산업명','시가총액']].tail(480)
joined

,순위,종목코드,종목명,현재가,거래량,시가총액,상장주식수,시장,회사명,산업코드,...,유동자산,부채총계,자본총계,매출액,매출원가,매출총이익,당기순이익,영업현금흐름,유동비율,부채비율
0,1,005930,삼성전자,49000,14206216,292519344950000,5969782550,KS,삼성전자,32600,...,1.813853e+11,89684076000,262880421000,2.304009e+11,1.472395e+11,8.316133e+10,21738865000,4.538292e+10,284.379742,34.115921
1,2,000660,SK하이닉스,82300,3344109,59914594639500,728002365,KS,SK하이닉스,32600,...,1.445760e+10,16846299000,47943195000,2.699073e+10,1.882528e+10,8.165458e+09,2016391000,6.483188e+09,183.611143,35.138040
2,3,207940,삼성바이오로직스,542000,539127,35861430000000,66165000,KS,삼성바이오로직스,32100,...,1.356262e+09,1557172767,4354453745,7.015919e+08,5.095844e+08,1.920074e+08,202904099,1.015362e+07,NaN,NaN
3,5,068270,셀트리온,216000,1273985,27722869704000,128346619,KS,셀트리온,32100,...,1.787340e+09,986733418,2906958188,1.128460e+09,4.984397e+08,6.300202e+08,297969203,4.739939e+08,271.952220,33.943846
4,6,035420,NAVER,168500,520725,27678382057500,164263395,KS,NAVER,106300,...,5.643714e+09,5795601052,6503926069,6.593400e+09,NaN,6.593400e+09,396821062,1.356843e+09,149.576907,89.109270
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1780,2271,054410,케이피티유,3020,13086,15281200000,5060000,KQ,케이피티유,32500,...,1.180718e+07,29344083,32888826,2.426961e+07,2.122144e+07,3.048165e+06,-764474,2.484600e+06,NaN,NaN
1781,2276,096870,엘디티,2200,21496,14692480000,6678400,KQ,엘디티,32600,...,5.667297e+06,1943262,9881656,6.955660e+06,5.057515e+06,1.898145e+06,-1398661,1.221650e+05,NaN,NaN
1782,2278,025890,한국주강,1290,7715,14588944110,11309259,KS,한국주강,32900,...,1.283543e+07,16295373,32212273,3.627140e+07,3.844710e+07,-2.175700e+06,-13564456,3.071580e+05,NaN,NaN
1783,2296,019490,하이트론,2305,27489,12748344175,5530735,KS,하이트론,32600,...,3.383360e+07,38483318,17097678,3.469537e+07,3.819625e+07,-3.500882e+06,-17825443,-8.217431e+06,126.450375,225.079207


In [16]:
joined['PER'] = joined['시가총액'] / joined['당기순이익'] / 1000
joined['PSR'] = joined['시가총액'] / joined['매출액'] / 1000
joined['PCR'] = joined['시가총액'] / joined['영업현금흐름'] / 1000
joined['PBR'] = joined['시가총액'] / joined['자본총계'] / 1000
joined['ROA'] = joined['당기순이익'] / joined['자산총계']
joined['GPA'] = joined['매출총이익'] / joined['자산총계']

In [30]:
low20 = joined.tail(int(len(reports) * 0.2))

In [31]:
low20 = low20[low20['당기순이익'] > 0]
low20['PER_Rank'] = low20['PER'].rank(method='max')
low20['PSR_Rank'] = low20['PSR'].rank(method='max')
low20['PCR_Rank'] = low20['PCR'].rank(method='max')
low20['PBR_Rank'] = low20['PBR'].rank(method='max')
low20['ROA_Rank'] = low20['ROA'].rank(method='max', ascending=False)
low20['GPA_Rank'] = low20['GPA'].rank(method='max', ascending=False)
low20['s_value'] = low20['PER_Rank'] + low20['PSR_Rank'] + low20['PCR_Rank'] + low20['PBR_Rank'] # 슈퍼가치전략
low20['PBRGPA'] = low20['PBR_Rank'] + low20['GPA_Rank'] # 신마법공식
low20['s_v_m'] = low20['PER_Rank'] + low20['PSR_Rank'] + low20['PBR_Rank'] + low20['GPA_Rank'] # 슈퍼밸류모멘텀

In [32]:
# 슈퍼가치전략
super_values = low20.sort_values(['s_value']).head(50)
print(len(super_values))
super_values.to_excel("../output/슈퍼가치전략_%s.xlsx"%price_date)

50


In [33]:
# 그래이엄 마지막 선물
graham = low20[low20['부채비율'] < 50]
graham = graham[graham['PER'] < 10]
print(len(graham))
graham.to_excel("../output/그레이엄의마지막선물_%s.xlsx"%price_date)

23


In [34]:
# NCAV 전략 - 뭔가 오류.... ㅠ.ㅠ
ncav = joined[(joined['유동자산'] - joined['부채총계']) > (joined['시가총액'] / 1000 * 1.5)]
ncav = ncav[ncav['당기순이익'] > 0]
print(len(ncav))
ncav.to_excel("../output/NCAV_%s.xlsx"%price_date)

32


In [35]:
# 그레이엄 마지막 선물 업그레이드
graham2 =low20[low20['부채비율'] < 50]
graham2 = graham[graham['ROA'] < 0.5]
graham2 = graham[graham['PBR'] >= 0.2]
print(len(graham2))
graham2 = graham2.sort_values(['PBR_Rank']).head(30)
graham2.to_excel("../output/그레이엄의마지막선물업그레이드_%s.xlsx"%price_date)

21


In [36]:
# 신마법공식 2.0
magic = low20.sort_values(['PBRGPA']).head(30)
print(len(magic))
magic.to_excel("../output/신마법공식2_%s.xlsx"%price_date)

30


In [37]:
# 슈퍼밸류모멘텀
svm = low20.sort_values(['s_v_m']).head(50)
print(len(svm))
year_ago = pd.read_csv('../data/stock_prices/20190412.csv')
year_ago['1년전주가'] = year_ago['현재가'].str.replace(",", "").astype(np.int64)
year_ago = year_ago[['종목코드', '1년전주가']]
svm = pd.merge(svm, year_ago, on='종목코드')
svm['주가Gap'] = svm['현재가'] - svm['1년전주가']
svm.head()
svm.to_excel("../output/슈퍼밸류모멘텀_%s.xlsx"%price_date)

50


In [ ]:
low20.keys()

In [38]:
svm[svm['주가Gap'] > 0]

,순위,종목코드,종목명,현재가,거래량,시가총액,상장주식수,시장,회사명,산업코드,...,PSR_Rank,PCR_Rank,PBR_Rank,ROA_Rank,GPA_Rank,s_value,PBRGPA,s_v_m,1년전주가,주가Gap
23,1740,032750,삼진,8160,52947,48960000000,6000000,KQ,삼진,32600,...,88.0,100.0,111.0,6.0,38.0,313.0,149.0,251.0,7150,1010


In [27]:
len(joined)

1785

In [28]:
len(reports)

2096

In [29]:
int(len(reports) * 0.2)

419